In [41]:
pip install tqdm


Note: you may need to restart the kernel to use updated packages.


In [7]:
import sys
print(sys.executable)


D:\bocci\anaconda3\envs\sanremo-env\python.exe


In [3]:
import os
import re
import numpy as np
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

# Download dei pacchetti necessari di NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Funzione di caricamento dei testi
main_folder = "C:/Users/bocci/OneDrive/Desktop/ESAME FINALE LOOKER STUDIO/Testi brani Sanremo 1951-2025"

def load_texts():
    texts = {}
    for year in range(1951, 2026):
        folder = os.path.join(main_folder, f"Testi_Sanremo_{year}")
        texts[year] = []
        for filename in os.listdir(folder):
            if filename.endswith(".txt"):
                with open(os.path.join(folder, filename), 'r', encoding='utf-8') as file:
                    texts[year].append(file.read())
    return texts

texts = load_texts()

# Lista di parole comuni da rimuovere (stopwords personalizzate)
custom_stopwords = set([
    "son", "quando", "già", "quel", "così", "poi", "questa", "quella", "questo", "quello", "anche", "ora", 
    "adesso", "è", "la", "il", "e", "di", "un", "una", "per", "che", "questi", "queste", "nessuno", "alcuni", 
    "alcune", "si", "c'è", "dove", "come", "allora", "bene", "male", "tanto", "tanto", "più", "meno", "tutti"
])

# Funzione di preprocessing dei testi con gestione dell'apostrofo
def preprocess_with_apostrophe_handling(text):
    # Rimuove i caratteri non alfabetici tranne gli apostrofi
    text = re.sub(r'[^a-zA-Zàèéìòóùù\s\'’]', '', text.lower())
    
    # Gestisce le parole con apostrofo, rimuovendo la lettera prima dell'apostrofo
    text = re.sub(r"\b(\w)\'", '', text)  # Rimuove la lettera prima dell'apostrofo (come "l'amore" -> "amore")
    
    # Tokenizzazione e rimozione delle stopwords
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('italian')).union(custom_stopwords)
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]  # Evita parole corte
    return tokens

processed_texts = {year: [preprocess_with_apostrophe_handling(text) for text in texts[year]] for year in texts}

# Funzione per calcolare la lunghezza dei testi
def text_lengths(year_texts):
    return [len(text) for text in year_texts]

lengths_per_year = {year: text_lengths(processed_texts[year]) for year in processed_texts}

mean_lengths = {year: np.mean(lengths_per_year[year]) for year in lengths_per_year}
median_lengths = {year: np.median(lengths_per_year[year]) for year in lengths_per_year}
min_lengths = {year: np.min(lengths_per_year[year]) for year in lengths_per_year}
max_lengths = {year: np.max(lengths_per_year[year]) for year in lengths_per_year}

# Salvataggio dei dati in un file CSV e XLSX
lengths_df = pd.DataFrame({
    "Anno": list(mean_lengths.keys()),
    "Lunghezza Media": list(mean_lengths.values()),
    "Lunghezza Mediana": list(median_lengths.values()),
    "Lunghezza Minima": list(min_lengths.values()),
    "Lunghezza Massima": list(max_lengths.values())
})

lengths_df.to_csv("sanremo_lunghezze_testi.csv", index=False)
lengths_df.to_excel("sanremo_lunghezze_testi.xlsx", index=False)

# Conteggio delle parole più comuni che compaiono in più di un testo
def get_most_common_words(year_texts):
    # Conta le parole per ogni anno
    word_counts = Counter([word for text in year_texts for word in text])
    
    # Considera solo le parole che appaiono in più di un testo
    filtered_words = {word: count for word, count in word_counts.items() if sum(1 for text in year_texts if word in text) > 1}
    
    # Restituisce le 10 parole più comuni
    return Counter(filtered_words).most_common(10)

common_words_per_year = {year: get_most_common_words(processed_texts[year]) for year in processed_texts}

# Salvataggio delle parole comuni in CSV
common_words_data = []
for year, common_words in common_words_per_year.items():
    for word, freq in common_words:
        common_words_data.append([year, word, freq])

common_words_df = pd.DataFrame(common_words_data, columns=["Anno", "Parola", "Frequenza"])
common_words_df.to_csv("sanremo_parole_comuni.csv", index=False)
common_words_df.to_excel("sanremo_parole_comuni.xlsx", index=False)

# Funzione per calcolare le parole più comuni per decennio e venticinquennio
def get_most_common_for_periods(period_texts, period_label):
    # Conta le parole per il periodo specificato
    word_counts = Counter([word for text in period_texts for word in text])
    
    # Considera solo le parole che appaiono in più di un testo
    filtered_words = {word: count for word, count in word_counts.items() if sum(1 for text in period_texts if word in text) > 1}
    
    # Restituisce le 10 parole più comuni come tuple (period_label, parola, frequenza)
    return [(period_label, word, count) for word, count in Counter(filtered_words).most_common(10)]

# Raccolta delle parole più comuni per decennio
decade_words = []
decades = [(1951, 1960), (1961, 1970), (1971, 1980), (1981, 1990), (1991, 2000), (2001, 2010), (2011, 2020), (2021, 2025)]
for start, end in decades:
    period_texts = [processed_texts[year] for year in range(start, end + 1)]
    combined_texts = [text for sublist in period_texts for text in sublist]
    decade_words.extend(get_most_common_for_periods(combined_texts, f"{start}-{end}"))

# Raccolta delle parole più comuni per venticinquennio
quarter_words = []
quarters = [(1951, 1975), (1976, 2000), (2001, 2025)]
for start, end in quarters:
    period_texts = [processed_texts[year] for year in range(start, end + 1)]
    combined_texts = [text for sublist in period_texts for text in sublist]
    quarter_words.extend(get_most_common_for_periods(combined_texts, f"{start}-{end}"))

# Creazione dei DataFrame per decennio e venticinquennio
decade_df = pd.DataFrame(decade_words, columns=["Periodo", "Parola", "Frequenza"])
quarter_df = pd.DataFrame(quarter_words, columns=["Periodo", "Parola", "Frequenza"])

# Salvataggio dei risultati in CSV e XLSX
decade_df.to_csv("sanremo_parole_comuni_decennio.csv", index=False)
decade_df.to_excel("sanremo_parole_comuni_decennio.xlsx", index=False)

quarter_df.to_csv("sanremo_parole_comuni_venticinquennio.csv", index=False)
quarter_df.to_excel("sanremo_parole_comuni_venticinquennio.xlsx", index=False)

print("📊 Tutti i dati sono stati salvati con successo!")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bocci\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bocci\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


📊 Tutti i dati sono stati salvati con successo!


In [11]:
import os
import re
import numpy as np
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from tqdm import tqdm  # Per il monitoraggio dell'avanzamento

# Download dei pacchetti necessari di NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Funzione di caricamento dei testi
main_folder = "C:/Users/bocci/OneDrive/Desktop/ESAME FINALE LOOKER STUDIO/Testi brani Sanremo 1951-2025"

def load_texts():
    texts = {}
    for year in range(1951, 2026):
        folder = os.path.join(main_folder, f"Testi_Sanremo_{year}")
        texts[year] = []
        for filename in os.listdir(folder):
            if filename.endswith(".txt"):
                with open(os.path.join(folder, filename), 'r', encoding='utf-8') as file:
                    texts[year].append(file.read())
    return texts

texts = load_texts()

# Parole chiave per le categorie (anatomia, natura, tecnologia)
anatomy_keywords = set([ 
    "cuore", "polmoni", "stomaco", "testa", "mani", "piedi", "occhi", "braccia", "gambe", "schiena", "pelle", 
    "dita", "bocca", "denti", "orecchie", "cervello", "nervi", "vene", "arterie", "organi", "corpo", "ossa", 
    "sangue", "respirare", "battere", "muovere", "camminare", "correre", "pensare", "vedere", "sentire", "toccare", 
    "gustare", "ascoltare", "dormire", "mangiare", "parlare", "piangere"
])

nature_keywords = set([
    "sole", "luna", "cielo", "stella", "nuvole", "mare", "fiume", "montagna", "collina", "valle", "foresta", 
    "alberi", "erba", "sabbia", "terra", "vento", "pioggia", "neve", "luce", "ombra", "ghiaccio", "fiori", "piante", 
    "uccelli", "pesci", "cani", "gatti", "cavalli", "api", "farfalle", "insetti", "bosco", "fauna", "flora", "natura"
])

tech_keywords = set([
    "internet", "computer", "rete", "software", "hardware", "digitale", "social", "smartphone", "applicazione", 
    "programma", "sito", "cloud", "server", "codice", "algoritmo", "byte", "dati", "pixel", "schermo", "monitor", 
    "robot", "automazione", "intelligenza artificiale", "vr", "ar", "droni", "macchina", "internet delle cose", "iot", 
    "smart", "instagram", "tik tok", "reel", "reels", "storie", "foto", "video"
])

# Lista di parole comuni da rimuovere (stopwords generali)
custom_stopwords = set([
    "son", "quando", "già", "quel", "così", "poi", "questa", "quella", "questo", "quello", "anche", "ora", 
    "adesso", "è", "la", "il", "e", "di", "un", "una", "per", "che", "questi", "queste", "nessuno", "alcuni", 
    "alcune", "si", "c'è", "dove", "come", "allora", "bene", "male", "tanto", "più", "meno", "tutti"
])

# Funzione di preprocessing dei testi con gestione dell'apostrofo
def preprocess_with_apostrophe_handling(text):
    # Rimuove i caratteri non alfabetici tranne gli apostrofi
    text = re.sub(r'[^a-zA-Zàèéìòóùù\s\'’]', '', text.lower())
    
    # Gestisce le parole con apostrofo, rimuovendo la lettera prima dell'apostrofo
    text = re.sub(r"\b(\w)\'", '', text)  # Rimuove la lettera prima dell'apostrofo (come "l'amore" -> "amore")
    
    # Tokenizzazione e rimozione delle stopwords
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('italian')).union(custom_stopwords)
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]  # Evita parole corte
    return tokens

processed_texts = {year: [preprocess_with_apostrophe_handling(text) for text in texts[year]] for year in texts}

# Funzione per calcolare le parole più comuni, considerando solo quelle presenti in più di una canzone
def get_most_common_words(year_texts):
    word_counts = Counter([word for text in year_texts for word in text])
    filtered_words = {word: count for word, count in word_counts.items() if sum(1 for text in year_texts if word in text) > 1}
    return Counter(filtered_words).most_common(10)

common_words_per_year = {year: get_most_common_words(processed_texts[year]) for year in processed_texts}

# Funzione per estrarre le parole chiave nelle categorie anatomia, natura e tecnologia
def extract_keywords_from_text(texts, keywords_set):
    flat_texts = [word for text in texts for word in text]  # Appiattisce la lista di testi
    return [word for word in flat_texts if word in keywords_set]

# Estrazione delle parole chiave per anatomia, natura e tecnologia
all_texts = [text for sublist in processed_texts.values() for text in sublist]

anatomy_words = extract_keywords_from_text(all_texts, anatomy_keywords)
nature_words = extract_keywords_from_text(all_texts, nature_keywords)
tech_words = extract_keywords_from_text(all_texts, tech_keywords)

# Funzione per calcolare le parole più comuni per decennio e venticinquennio
def get_most_common_for_periods(period_texts, period_label):
    word_counts = Counter([word for text in period_texts for word in text])
    filtered_words = {word: count for word, count in word_counts.items() if sum(1 for text in period_texts if word in text) > 1}
    return [(period_label, word, count) for word, count in Counter(filtered_words).most_common(10)]

# Raccolta delle parole comuni per decennio e venticinquennio
decade_words = []
decades = [(1951, 1960), (1961, 1970), (1971, 1980), (1981, 1990), (1991, 2000), (2001, 2010), (2011, 2020), (2021, 2025)]

for start, end in tqdm(decades, desc="Elaborazione decenni"):
    period_texts = [processed_texts[year] for year in range(start, end + 1)]
    combined_texts = [text for sublist in period_texts for text in sublist]
    decade_words.extend(get_most_common_for_periods(combined_texts, f"{start}-{end}"))

quarter_words = []
quarters = [(1951, 1975), (1976, 2000), (2001, 2025)]

for start, end in tqdm(quarters, desc="Elaborazione venticinquennio"):
    period_texts = [processed_texts[year] for year in range(start, end + 1)]
    combined_texts = [text for sublist in period_texts for text in sublist]
    quarter_words.extend(get_most_common_for_periods(combined_texts, f"{start}-{end}"))

# Creazione dei DataFrame per decennio e venticinquennio
decade_df = pd.DataFrame(decade_words, columns=["Periodo", "Parola", "Frequenza"])
quarter_df = pd.DataFrame(quarter_words, columns=["Periodo", "Parola", "Frequenza"])

# Creazione dei DataFrame per le categorie
anatomy_df = pd.DataFrame(Counter(anatomy_words).items(), columns=["Parola", "Frequenza"])
nature_df = pd.DataFrame(Counter(nature_words).items(), columns=["Parola", "Frequenza"])
tech_df = pd.DataFrame(Counter(tech_words).items(), columns=["Parola", "Frequenza"])

# Creazione di un unico file Excel con 3 fogli
with pd.ExcelWriter("sanremo_parole_per_categoria.xlsx") as writer:
    anatomy_df.to_excel(writer, sheet_name="Anatomia", index=False)
    nature_df.to_excel(writer, sheet_name="Natura", index=False)
    tech_df.to_excel(writer, sheet_name="Tecnologia", index=False)

# Creazione dei file per il conteggio delle parole comuni
common_words_data = []
for year, common_words in common_words_per_year.items():
    for word, freq in common_words:
        common_words_data.append([year, word, freq])

common_words_df = pd.DataFrame(common_words_data, columns=["Anno", "Parola", "Frequenza"])

# Salvataggio delle parole comuni in un file Excel
with pd.ExcelWriter("sanremo_parole_comuni.xlsx") as writer:
    common_words_df.to_excel(writer, index=False)

# Salvataggio dei dati per decennio e venticinquennio
with pd.ExcelWriter("sanremo_parole_comuni_decennio_e_venticinquennio.xlsx") as writer:
    decade_df.to_excel(writer, sheet_name="Decenni", index=False)
    quarter_df.to_excel(writer, sheet_name="Venticinquennio", index=False)

print("📊 Tutti i dati sono stati salvati con successo!")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bocci\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bocci\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Elaborazione venticinquennio: 100%|██████████| 3/3 [00:53<00:00, 17.96s/it]

📊 Tutti i dati sono stati salvati con successo!


In [9]:
import os
import re
import numpy as np
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

# Download dei pacchetti necessari di NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Funzione di caricamento dei testi
main_folder = "C:/Users/bocci/OneDrive/Desktop/ESAME FINALE LOOKER STUDIO/Testi brani Sanremo 1951-2025"

def load_texts():
    texts = {}
    for year in range(1951, 2026):
        folder = os.path.join(main_folder, f"Testi_Sanremo_{year}")
        texts[year] = []
        for filename in os.listdir(folder):
            if filename.endswith(".txt"):
                with open(os.path.join(folder, filename), 'r', encoding='utf-8') as file:
                    texts[year].append(file.read())
    return texts

texts = load_texts()

# Lista di parole comuni da rimuovere (stopwords personalizzate)
custom_stopwords = set([
    "son", "quando", "già", "quel", "così", "poi", "questa", "quella", "questo", "quello", "anche", "ora", 
    "adesso", "è", "la", "il", "e", "di", "un", "una", "per", "che", "questi", "queste", "nessuno", "alcuni", 
    "alcune", "si", "c'è", "dove", "come", "allora", "bene", "male", "tanto", "tanto", "più", "meno", "tutti"
])

# Funzione di preprocessing dei testi con gestione dell'apostrofo
def preprocess_with_apostrophe_handling(text):
    # Rimuove i caratteri non alfabetici tranne gli apostrofi
    text = re.sub(r'[^a-zA-Zàèéìòóùù\s\'’]', '', text.lower())
    
    # Gestisce le parole con apostrofo, rimuovendo la lettera prima dell'apostrofo
    text = re.sub(r"\b(\w)\'", '', text)  # Rimuove la lettera prima dell'apostrofo (come "l'amore" -> "amore")
    
    # Tokenizzazione e rimozione delle stopwords
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('italian')).union(custom_stopwords)
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]  # Evita parole corte
    return tokens

processed_texts = {year: [preprocess_with_apostrophe_handling(text) for text in texts[year]] for year in texts}

# Funzione per calcolare la lunghezza dei testi
def text_lengths(year_texts):
    return [len(text) for text in year_texts]

lengths_per_year = {year: text_lengths(processed_texts[year]) for year in processed_texts}

mean_lengths = {year: np.mean(lengths_per_year[year]) for year in lengths_per_year}
median_lengths = {year: np.median(lengths_per_year[year]) for year in lengths_per_year}
min_lengths = {year: np.min(lengths_per_year[year]) for year in lengths_per_year}
max_lengths = {year: np.max(lengths_per_year[year]) for year in lengths_per_year}

# Creazione del DataFrame per la lunghezza dei testi
lengths_df = pd.DataFrame({
    "Anno": list(mean_lengths.keys()),
    "Lunghezza Media": list(mean_lengths.values()),
    "Lunghezza Mediana": list(median_lengths.values()),
    "Lunghezza Minima": list(min_lengths.values()),
    "Lunghezza Massima": list(max_lengths.values())
})

lengths_df.to_excel("sanremo_lunghezze_testi.xlsx", index=False)

# Lista di categorie di parole (anatomia, natura, tecnologia)
categories = {
    "anatomia": ["cuore", "testa", "braccio", "mano", "occhio", "bocca", "gamba", "schiena", "stomaco", "collo"],
    "natura": ["albero", "fiore", "acqua", "mare", "montagna", "cielo", "sole", "terra", "luna", "stella"],
    "tecnologia": ["computer", "internet", "smartphone", "tecnologia", "robot", "digitale", "software", "hardware", "rete", "app"]
}

# Funzione per calcolare le parole più comuni che compaiono in più di una canzone per categoria
def get_most_common_words_for_category(year_texts, category):
    word_counts = Counter([word for text in year_texts for word in text if word in category])
    
    # Considera solo le parole che appaiono in più di un testo
    filtered_words = {word: count for word, count in word_counts.items() if sum(1 for text in year_texts if word in text) > 1}
    
    return Counter(filtered_words).most_common(10)

# Funzione per ottenere parole comuni per periodi (decennio, venticinquennio)
def get_most_common_for_periods(period_texts, period_label, category):
    word_counts = Counter([word for text in period_texts for word in text if word in category])
    
    # Considera solo le parole che appaiono in più di un testo
    filtered_words = {word: count for word, count in word_counts.items() if sum(1 for text in period_texts if word in text) > 1}
    
    return [(period_label, word, count) for word, count in Counter(filtered_words).most_common(10)]

# Funzione per raccogliere parole comuni per categoria su tutti gli anni
def collect_category_words_for_all_years():
    category_words_data = []
    for category_name, category_words in categories.items():
        for year, year_texts in processed_texts.items():
            common_words = get_most_common_words_for_category(year_texts, category_words)
            for word, freq in common_words:
                category_words_data.append([year, category_name, word, freq])
    return category_words_data

# Raccolta delle parole comuni per categoria
category_words_data = collect_category_words_for_all_years()

# Creazione del DataFrame e salvataggio in Excel
category_words_df = pd.DataFrame(category_words_data, columns=["Anno", "Categoria", "Parola", "Frequenza"])
category_words_df.to_excel("sanremo_parole_comuni_per_categoria.xlsx", index=False)

print("📊 Tutti i dati sono stati salvati con successo!")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bocci\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bocci\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


📊 Tutti i dati sono stati salvati con successo!


In [15]:
import os
import re
import numpy as np
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

# Download dei pacchetti necessari di NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Funzione di caricamento dei testi
main_folder = "C:/Users/bocci/OneDrive/Desktop/ESAME FINALE LOOKER STUDIO/Testi brani Sanremo 1951-2025"

def load_texts():
    texts = {}
    for year in range(1951, 2026):
        folder = os.path.join(main_folder, f"Testi_Sanremo_{year}")
        texts[year] = []
        for filename in os.listdir(folder):
            if filename.endswith(".txt"):
                with open(os.path.join(folder, filename), 'r', encoding='utf-8') as file:
                    texts[year].append(file.read())
    return texts

texts = load_texts()

# Liste di parole comuni da rimuovere (stopwords personalizzate)
anatomy_keywords = set([
    "cuore", "polmoni", "stomaco", "testa", "mani", "piedi", "occhi", "braccia", "gambe", "schiena", "pelle", "dita", "bocca", "denti", "orecchie", "cervello", "nervi", "vene", "arterie", "organi", "corpo", "ossa", "sangue",
    "respirare", "battere", "muovere", "camminare", "correre", "pensare", "vedere", "sentire", "toccare", "gustare", "ascoltare", "dormire", "mangiare", "parlare", "piangere"
])

nature_keywords = set([
    "sole", "luna", "cielo", "stella", "nuvole", "mare", "fiume", "montagna", "collina", "valle", "foresta", "alberi", "erba", "sabbia", "terra", "vento", "pioggia", "neve", "luce", "ombra", "ghiaccio",
    "fiori", "piante", "uccelli", "pesci", "cani", "gatti", "cavalli", "api", "farfalle", "insetti", "bosco", "fauna", "flora", "natura"
])

tech_keywords = set([
    "internet", "computer", "rete", "software", "hardware", "digitale", "social", "smartphone", "applicazione", "programma", "sito", "cloud", "server", "codice", "algoritmo", "byte", "dati", "pixel", "schermo", "monitor",
    "robot", "automazione", "intelligenza artificiale", "vr", "ar", "droni", "macchina", "internet delle cose", "iot", "smart",
    "instagram", "tiktok", "reels", "storiedistagram", "foto", "video", "post", "like", "followers", "hashtag", "socialnetwork"
])

# Lista di parole comuni da rimuovere (stopwords generali)
custom_stopwords = set([
    "son", "quando", "già", "quel", "così", "poi", "questa", "quella", "questo", "quello", "anche", "ora", 
    "adesso", "è", "la", "il", "e", "di", "un", "una", "per", "che", "questi", "queste", "nessuno", "alcuni", 
    "alcune", "si", "c'è", "dove", "come", "allora", "bene", "male", "tanto", "più", "meno", "tutti"
])

# Funzione di preprocessing dei testi con gestione dell'apostrofo
def preprocess_with_apostrophe_handling(text):
    # Rimuove i caratteri non alfabetici tranne gli apostrofi
    text = re.sub(r'[^a-zA-Zàèéìòóùù\s\'’]', '', text.lower())
    
    # Gestisce le parole con apostrofo, rimuovendo la lettera prima dell'apostrofo
    text = re.sub(r"\b(\w)\'", '', text)  # Rimuove la lettera prima dell'apostrofo (come "l'amore" -> "amore")
    
    # Tokenizzazione e rimozione delle stopwords
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('italian')).union(custom_stopwords)
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]  # Evita parole corte
    return tokens

processed_texts = {year: [preprocess_with_apostrophe_handling(text) for text in texts[year]] for year in texts}

# Funzione per contare le occorrenze di parole per categoria
def count_category_keywords(texts, category_keywords):
    counts = Counter()
    for text in texts:
        counts.update([word for word in text if word in category_keywords])
    return counts

# Funzione per ottenere le parole più comuni per periodo
def get_most_common_for_periods(period_texts, period_label):
    word_counts = Counter([word for text in period_texts for word in text])
    filtered_words = {word: count for word, count in word_counts.items() if sum(1 for text in period_texts if word in text) > 1}
    
    # Usa Counter per ordinare e ottenere le parole più comuni
    filtered_counter = Counter(filtered_words)
    return [(period_label, word, count) for word, count in filtered_counter.most_common(10)]

# Raccolta delle parole più comuni per decennio
decade_words = []
for start, end in [(1951, 1960), (1961, 1970), (1971, 1980), (1981, 1990), (1991, 2000), (2001, 2010), (2011, 2020), (2021, 2025)]:
    period_texts = [processed_texts[year] for year in range(start, end + 1)]
    combined_texts = [text for sublist in period_texts for text in sublist]
    decade_words.extend(get_most_common_for_periods(combined_texts, f"{start}-{end}"))

# Raccolta delle parole più comuni per venticinquennio
quarter_words = []
for start, end in [(1951, 1975), (1976, 2000), (2001, 2025)]:
    period_texts = [processed_texts[year] for year in range(start, end + 1)]
    combined_texts = [text for sublist in period_texts for text in sublist]
    quarter_words.extend(get_most_common_for_periods(combined_texts, f"{start}-{end}"))

# Calcolo delle occorrenze per le categorie anatomiche, naturali e tecnologiche
anatomy_counts_per_year = {year: count_category_keywords(processed_texts[year], anatomy_keywords) for year in processed_texts}
nature_counts_per_year = {year: count_category_keywords(processed_texts[year], nature_keywords) for year in processed_texts}
tech_counts_per_year = {year: count_category_keywords(processed_texts[year], tech_keywords) for year in processed_texts}

# Aggiungi i conteggi nel DataFrame per decennio e venticinquennio
def add_category_counts(df, period, category_counts):
    for category, counts in category_counts.items():
        df.loc[df['Periodo'] == period, category] = sum(counts.values())

# Creazione del DataFrame per decennio e venticinquennio
decade_df = pd.DataFrame(decade_words, columns=["Periodo", "Parola", "Frequenza"])
quarter_df = pd.DataFrame(quarter_words, columns=["Periodo", "Parola", "Frequenza"])

# Aggiungi i conteggi delle categorie per decennio
for start, end in [(1951, 1960), (1961, 1970), (1971, 1980), (1981, 1990), (1991, 2000), (2001, 2010), (2011, 2020), (2021, 2025)]:
    period = f"{start}-{end}"
    anatomy_counts = {year: anatomy_counts_per_year[year] for year in range(start, end + 1)}
    nature_counts = {year: nature_counts_per_year[year] for year in range(start, end + 1)}
    tech_counts = {year: tech_counts_per_year[year] for year in range(start, end + 1)}
    
    add_category_counts(decade_df, period, anatomy_counts)
    add_category_counts(decade_df, period, nature_counts)
    add_category_counts(decade_df, period, tech_counts)

# Aggiungi i conteggi delle categorie per venticinquennio
for start, end in [(1951, 1975), (1976, 2000), (2001, 2025)]:
    period = f"{start}-{end}"
    anatomy_counts = {year: anatomy_counts_per_year[year] for year in range(start, end + 1)}
    nature_counts = {year: nature_counts_per_year[year] for year in range(start, end + 1)}
    tech_counts = {year: tech_counts_per_year[year] for year in range(start, end + 1)}
    
    add_category_counts(quarter_df, period, anatomy_counts)
    add_category_counts(quarter_df, period, nature_counts)
    add_category_counts(quarter_df, period, tech_counts)

# Salvataggio dei risultati in XLSX
with pd.ExcelWriter("sanremo_parole_comuni_decennio.xlsx") as writer:
    # Creazione di fogli separati per ogni categoria
    anatomy_df = decade_df[['Periodo', 'Parola', 'Frequenza']]
    anatomy_df = anatomy_df[anatomy_df['Parola'].isin(anatomy_keywords)]
    anatomy_df.to_excel(writer, sheet_name="Anatomia", index=False)

    nature_df = decade_df[['Periodo', 'Parola', 'Frequenza']]
    nature_df = nature_df[nature_df['Parola'].isin(nature_keywords)]
    nature_df.to_excel(writer, sheet_name="Natura", index=False)

    tech_df = decade_df[['Periodo', 'Parola', 'Frequenza']]
    tech_df = tech_df[tech_df['Parola'].isin(tech_keywords)]
    tech_df.to_excel(writer, sheet_name="Tecnologia", index=False)

with pd.ExcelWriter("sanremo_parole_comuni_quarter.xlsx") as writer:
    # Creazione di fogli separati per ogni categoria
    anatomy_df = quarter_df[['Periodo', 'Parola', 'Frequenza']]
    anatomy_df = anatomy_df[anatomy_df['Parola'].isin(anatomy_keywords)]
    anatomy_df.to_excel(writer, sheet_name="Anatomia", index=False)

    nature_df = quarter_df[['Periodo', 'Parola', 'Frequenza']]
    nature_df = nature_df[nature_df['Parola'].isin(nature_keywords)]
    nature_df.to_excel(writer, sheet_name="Natura", index=False)

    tech_df = quarter_df[['Periodo', 'Parola', 'Frequenza']]
    tech_df = tech_df[tech_df['Parola'].isin(tech_keywords)]
    tech_df.to_excel(writer, sheet_name="Tecnologia", index=False)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bocci\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bocci\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
